In [3]:
from Lutil.checkpoints import InlineCheckpoint
import numpy as np
import pandas as pd
from Lutil._exceptions import InlineEnvironmentWarning
import unittest


RM = "Runned."


def R():
    print(RM, end="")
    

def add_give_c(a, b):
    c = "reset"
    with InlineCheckpoint(watch=["a", "b"], produce=["c"], _id="add_give_c"):
        R()
        c = a + b
    return c

class Foo(object):
    pass

def add_give_c_in_obj(a, b):
    f = Foo()
    f.c = "reset"

    with InlineCheckpoint(watch=["a", "b"], produce=["f.c"], _id="add_give_c"):
        R()
        f.c = a + b

    return f.c

def watch_obj_value(a):
    f = Foo()
    f.a = a

    with InlineCheckpoint(watch=["f.a"], produce=["f.c"], _id="add_give_c"):
        R()
        f.c = a

    return f.c

def fail_1():
    with InlineCheckpoint(watch=["a"], produce=["b"], _id="fail_1"):
        R()
        b = a
    return b

def fail_2():
    f = Foo()
    with InlineCheckpoint(watch=["f.a"], produce=["b"], _id="fail_2"):
        R()
        b = f.a
    return b

def multi_level(a):
    f = Foo()
    f.f = Foo()
    f.f.f = Foo()
    f.f.f.a = a

    with InlineCheckpoint(watch=["f.f.f.a"], produce=["f.f.b"], _id="multi_level"):
        R()
        f.f.b = f.f.f.a
    return f.f.b

def default_id(a):
    f = Foo()
    f.a = a
    with InlineCheckpoint(watch=["f.a"], produce=["f.b"]):
        R()
        f.b = f.a
    return f.b

def no_watch():
    f = Foo()
    with InlineCheckpoint(watch=[], produce=["f.a"]):
        R()
        f.a = 1

    return f.a

def produce_two():
    f = Foo()
    with InlineCheckpoint(watch=[], produce=["f.a", "f.b"]):
        R()
        f.a = 1
        f.b = 2
    return f.a, f.b

class InlineCheckpointTest(unittest.TestCase):
    
    arr1 = np.array([
        [1],
        [2]
    ])

    arr2 = np.array([
        [1, 1],
        [2, 2]
    ])

    df1 = pd.DataFrame({
        "a": [1],
        "b": [2]
    })

    df2 = pd.DataFrame({
        "a": [1],
        "b": [2.1]
    })

    s1 = pd.Series([1, 2, 3, 4])
    
    arr3 = np.array([
        [1,2,4,7],
        [2,1,2,2],
    ])

    df3 = pd.DataFrame({
        "a": [1.12],
        "b": [2.1]
    })
    
    
    def setUp(self):
        self.M = io.StringIO()
        sys.stdout = self.M

    def clear(self):
        self.M.truncate(0)
        self.M.seek(0)

    def runned(self):
        self.assertEqual(self.M.getvalue(), RM)
        self.clear()

    def not_runned(self):
        self.assertEqual(self.M.getvalue(), "")
        self.clear()

    def tearDown(self):
        sys.stdout = sys.__stdout__
        dir_name = ".Lutil-checkpoint"
        sub_dir_name = os.path.join("tests", dir_name)
        if os.path.exists(dir_name):
            shutil.rmtree(dir_name)

        if os.path.exists(sub_dir_name):
            shutil.rmtree(sub_dir_name)

    def test_add_give_c(self):
        # with self.assertWarns(InlineEnvironmentWarning):
        self.assertEqual(add_give_c(1,2),1+2)
        self.runned()

        # with self.assertWarns(InlineEnvironmentWarning):
        self.assertEqual(add_give_c(1,3),1+3)
        self.runned()

        # with self.assertWarns(InlineEnvironmentWarning):
        self.assertEqual(add_give_c(1,2),1+2)
        self.runned()

        # with self.assertWarns(InlineEnvironmentWarning):
        self.assertEqual(add_give_c(1,3),1+3)
        self.runned()

    def test_add_give_c_in_obj(self):
        self.assertEqual(add_give_c_in_obj(1,2),1+2)
        self.runned()

        self.assertEqual(add_give_c_in_obj(1,3),1+3)
        self.runned()

        self.assertEqual(add_give_c_in_obj(1,2),1+2)
        self.not_runned()

        self.assertEqual(add_give_c_in_obj(1,3),1+3)
        self.not_runned()

    def test_add_give_c_in_obj_df(self):
        self.assertTrue(
            (add_give_c_in_obj(self.df1,self.df2)==self.df1+self.df2).all().all()
            )
        self.runned()

        self.assertTrue(
            (add_give_c_in_obj(self.df1,self.df3)==self.df1+self.df3).all().all()
            )
        self.runned()

        self.assertTrue(
            (add_give_c_in_obj(self.df1,self.df2)==self.df1+self.df2).all().all()
            )
        self.not_runned()

        self.assertTrue(
            (add_give_c_in_obj(self.df1,self.df3)==self.df1+self.df3).all().all()
            )
        self.not_runned()

    def test_add_give_c_in_obj_arr(self):
        self.assertTrue(
            (add_give_c_in_obj(self.arr1,self.arr2)==self.arr1+self.arr2).all()
            )
        self.runned()

        self.assertTrue(
            (add_give_c_in_obj(self.arr1,self.arr3)==self.arr1+self.arr3).all()
            )
        self.runned()

        self.assertTrue(
            (add_give_c_in_obj(self.arr1,self.arr2)==self.arr1+self.arr2).all()
            )
        self.not_runned()

        self.assertTrue(
            (add_give_c_in_obj(self.arr1,self.arr3)==self.arr1+self.arr3).all()
            )
        self.not_runned()

    def test_raise(self):
        with self.assertRaises(ValueError):
            fail_1()

        with self.assertRaises(ValueError):
            fail_2()

    def test_watch_obj_value(self):
        self.assertEqual(watch_obj_value(1),1)
        self.runned()

        self.assertEqual(watch_obj_value(2),2)
        self.runned()

        self.assertEqual(watch_obj_value(1),1)
        self.not_runned()

        self.assertEqual(watch_obj_value(2),2)
        self.not_runned()

    def test_multi_level(self):
        self.assertEqual(multi_level(1),1)
        self.runned()

        self.assertEqual(multi_level(2),2)
        self.runned()

        self.assertEqual(multi_level(1),1)
        self.not_runned()

        self.assertEqual(multi_level(2),2)
        self.not_runned()

    def test_default_id(self):
        self.assertEqual(default_id(1),1)
        self.runned()

        self.assertEqual(default_id(2),2)
        self.runned()

        self.assertEqual(default_id(1),1)
        self.not_runned()

        self.assertEqual(default_id(2),2)
        self.not_runned()

    def test_no_watch(self):
        self.assertEqual(no_watch(),1)
        self.runned()

        self.assertEqual(no_watch(),1)
        self.not_runned()

    def test_produce_two(self):
        self.assertSequenceEqual(produce_two(), (1,2))
        self.runned()

        self.assertSequenceEqual(produce_two(), (1,2))
        self.not_runned()


In [4]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(InlineCheckpointTest)
r = unittest.TextTestRunner().run(suite)
assert not r.failures

C:\Personal\gitkrakenRepo\learning-utility\venv\lib\site-packages\lutil-0.0.1.dev0-py3.7.egg\Lutil\IO\_checkpoint.py:204: InlineEnvironmentWarning: InlineCheckpoint is only functional in '__main__' to auto-cache a variable, while you are trying to use it inside a function or method.
The 'produce' will not be cached and the computation will always happen.
Please refer to https://learning-utility.readthedocs.io/en/latest/ for alternative solutions.
  warnings.warn(InlineEnvironmentWarning())
..........
----------------------------------------------------------------------
Ran 10 tests in 0.161s

OK
